## digit-recognizer 데이터를 이용하여 앙상블 해보자!

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
from sklearn.linear_model import LogisticRegression
#import mglearn 
from sklearn.preprocessing import MinMaxScaler

train_df = pd.read_csv("./data/digit-recognizer/train.csv")
test_df = pd.read_csv("./data/digit-recognizer/test.csv") 


In [3]:
# train, test 데이터 생성
# 정규화
scaler= MinMaxScaler()
#train_num = int(train_df.shape[0]*0.8) 
train_num = int(train_df.shape[0])

x_data = train_df.drop("label",axis=1,inplace=False)
x_data = scaler.fit_transform(x_data)

sess = tf.Session()
y_data = sess.run(tf.one_hot(train_df["label"], 10))

# 데이터 분할해 x data 생성 - 80:20
train_x_data = x_data[:train_num]  # nparray에서 자른 값 ->  뒤의 값이 exclusive 해서 16000개
test_x_data = x_data[train_num:]

# 학습용, 테스트용 y data 생성 -> one hot encoding

train_y_data = sess.run(tf.one_hot(train_df.loc[:train_num-1,"label"], 10)) # 원래 1차원 자료가 one-hot encoding 적용하면 2차원 자료가 된다.
# sess.run(tf.one_hot(bmi_df.loc[:train_num-1,"label"], 3)).shape # data frame이라 loc 사용하면 뒤부분 inclusive하다 -> train_num -1 빼줘야 함! 
test_y_data = sess.run(tf.one_hot(train_df.loc[train_num:,"label"], 10))


In [4]:

## Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()  # 그래프 reset

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다

# 2.2 Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 

L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                strides=1,activation=tf.nn.relu)  
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)

L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                strides=1, activation=tf.nn.relu) 
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

# 2.3 FC layer -> dense layer 
L2 = tf.reshape(L2, [-1,7*7*64])
dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
dense1 = tf.nn.dropout(dense1, rate=dropout_rate)  

dense2 = tf.layers.dense(inputs=dense1, units=512, activation=tf.nn.relu)
dense2 = tf.nn.dropout(dense2, rate=dropout_rate)

H = tf.layers.dense(inputs=dense2, units=10)  # H에서는 activation 쓰지 않음

# cost 
cost = tf.losses.softmax_cross_entropy(Y,H)

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())


# 학습 - epoch, batch 등 

for n in range(15):
    # 그래프 reset
    tf.reset_default_graph() 

    num_of_epoch = 30
    batch_size = 100

    # 모델은 생성됐으니까 H만 돌려보자.
    for step in range(num_of_epoch):
        num_of_iter = int(train_num/batch_size)
        cost_val = 0
        for i in range(num_of_iter):
            batch_x = train_x_data[batch_size*i:batch_size*(i+1)]
            batch_y = train_y_data[batch_size*i:batch_size*(i+1)]       
            _, cost_val,H_val = sess.run([train,cost,H],feed_dict = {X:batch_x,
                                                        Y:batch_y,
                                                        dropout_rate:0.3}) # 30% 끄고 학습해라
    print(cost_val)
    # test 데이터로 돌려주자 
    H_val = sess.run(H, feed_dict={X:test_x_data,
                                   dropout_rate:0.3})

    # 나온 H 값들을 각 자리마다 더해준다.
    if n==0:  # 초기 ndarray 지정
        H_result = H_val
    else:
        H_result = np.add(H_result,H_val)  # 각 자리의 합을 구해주는 함수
    

TypeError: dropout() got an unexpected keyword argument 'rate'

In [ ]:
# 학습이 종료되었으니 정확도를 측정하자! 

predict_result = np.argmax(H_result,1)
true_result = np.argmax(test_y_data,1)
correct = np.equal(predict_result,true_result)
accuracy = np.mean(correct)
print(accuracy)

## 전체를 학습해서 test데이터로 돌려보자!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
from sklearn.linear_model import LogisticRegression
#import mglearn 
from sklearn.preprocessing import MinMaxScaler

train_df = pd.read_csv("/content/gdrive/My Drive/python_DA/data/digit-recognizer/train.csv")
test_df = pd.read_csv("/content/gdrive/My Drive/python_DA/data/digit-recognizer/test.csv") 

# 학습용 데이터
scaler= MinMaxScaler()
#train_num = int(train_df.shape[0]*0.8) 
train_num = int(train_df.shape[0])

# train용 데이터
train_x_data = train_df.drop("label",axis=1,inplace=False)
train_x_data = scaler.fit_transform(train_x_data)

sess = tf.Session()
train_y_data = sess.run(tf.one_hot(train_df["label"], 10))

# test용 데이터
test_x_data = test_df
test_x_data = scaler.fit_transform(test_x_data)

In [ ]:
## Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()  # 그래프 reset

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다

# 2.2 Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 

L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                strides=1,activation=tf.nn.relu)  
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)

L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                strides=1, activation=tf.nn.relu) 
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

# 2.3 FC layer -> dense layer 
L2 = tf.reshape(L2, [-1,7*7*64])
dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
dense1 = tf.nn.dropout(dense1, rate=dropout_rate)  

dense2 = tf.layers.dense(inputs=dense1, units=512, activation=tf.nn.relu)
dense2 = tf.nn.dropout(dense2, rate=dropout_rate)

H = tf.layers.dense(inputs=dense2, units=10)  # H에서는 activation 쓰지 않음

# cost 
cost = tf.losses.softmax_cross_entropy(Y,H)

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())


# 학습 - epoch, batch 등 

for n in range(10):
    # 그래프 reset
    tf.reset_default_graph() 

    num_of_epoch = 30
    batch_size = 100

    # 모델은 생성됐으니까 H만 돌려보자.
    for step in range(num_of_epoch):
        num_of_iter = int(train_num/batch_size)
        cost_val = 0
        for i in range(num_of_iter):
            batch_x = train_x_data[batch_size*i:batch_size*(i+1)]
            batch_y = train_y_data[batch_size*i:batch_size*(i+1)]       
            _, cost_val,H_val = sess.run([train,cost,H],feed_dict = {X:batch_x,
                                                                     Y:batch_y,
                                                                     dropout_rate:0.3}) # 30% 끄고 학습해라
    #print(cost_val)
    # test 데이터로 돌려주자 
    H_val = sess.run(H, feed_dict={X:test_x_data,
                                   dropout_rate:0})

    # 나온 H 값들을 각 자리마다 더해준다.
    if n==0:  # 초기 ndarray 지정
        H_result = H_val
    else:
        H_result = np.add(H_result,H_val)  # 각 자리의 합을 구해주는 함수
    
predict_result = np.argmax(H_result,1)
print(predict_result)

In [ ]:
my_df = pd.DataFrame()
my_df["ImageId"] = range(1,test_df.shape[0]+1)
my_df["Label"] = predict_result
my_df


In [ ]:
my_df.to_csv("/content/gdrive/My Drive/python_DA/data/digit-recognizer/ensemble.csv",
                 sep=',',
                 na_rep='NaN', 
                 columns = ['ImageId','Label'], # columns to write
                 index = False) # do not write index